# Port (Socket)
In node graph programming, people use `sockets` to indicate the type of data that can be transferred from one node to another. This is similar to AiiDA’s `port`. We will use the name port to reuse the concepts already in AiiDA as much as possible. Their differences will be introduced later.

Usually, the ports are created automatically from an AiiDA component (e.g., WorkChain), or generated automatically based on the function arguments. There are also some built-in ports(sockets), like `_wait` and `_outputs`. 



In [11]:
from aiida_workgraph import node

@node.calcfunction()
def multiply(x, y):
   return x*y

print("Input ports: ", multiply.node().inputs.keys())
print("Output ports: ", multiply.node().outputs.keys())

Input ports:  ['x', 'y', '_wait']
Output ports:  ['result', '_wait', '_outputs']


If you want to change the name of the output ports, or if there are more than one output. You can define the outputs explicitly.

In [9]:
from aiida_workgraph import node

@node(outputs=[["General", "sum"],
               ["General", "difference"]])
def add_minus(x, y):
   return {"sum": x + y, "difference": x - y}


print("Input ports: ", add_minus.node().inputs.keys())
print("Ouput ports: ", add_minus.node().outputs.keys())

Input ports:  ['x', 'y', '_wait']
Ouput ports:  ['sum', 'difference', '_wait', '_outputs']


Two values are needed to define a port, e.g., `["General", "sum"]`, where the first value `General` indicates the data type, and the second value is the name of the port. We use `General` for any data type.

## Assign socket type based on typing hints
The type hints in the function signature can be used to assign the socket type. The following table shows the mapping between the type hints and the socket type.

| Type hint     | Socket type   |
|---------------|---------------|
| `orm.Int`     | `AiiDAInt`    |
| `orm.Str`     | `AiiDAString` |
| `orm.Float`   | `AiiDAFloat`  |
| `orm.Bool`    | `AiiDABool`   |


In [1]:
from aiida_workgraph import node

@node.calcfunction()
def add(x: int, y: float) -> float:
   return x + y

for input in add.node().inputs:
   print("{:30s}: {:20s}".format(input.name, input.identifier))

aiida_socket_maping:  {<class 'aiida.orm.nodes.data.int.Int'>: 'AiiDAInt', <class 'aiida.orm.nodes.data.float.Float'>: 'AiiDAFloat', <class 'aiida.orm.nodes.data.str.Str'>: 'AiiDAString', <class 'aiida.orm.nodes.data.bool.Bool'>: 'AiiDABool'}
metadata                      : General             
metadata.store_provenance     : General             
metadata.description          : General             
metadata.label                : General             
metadata.call_link_label      : General             
x                             : AiiDAInt            
y                             : AiiDAFloat          
_wait                         : General             




## Data validation (**Experimental**)
One can use the class of the data directly when defining the port.

**For the moment, data validation is experimentally supported.** Thus, I suggest you always use `General` for the port.

In [5]:
from aiida_workgraph import node
from aiida import orm

@node.calcfunction(
    inputs=[[orm.Int, "x"], [orm.Float, "y"]],
    outputs=[[orm.Float, "result"]]
)
def add(x, y):
    result = x + y
    return result

## Advanced concept of Socket

In the GUI of node graph programming, a socket is displayed as a circle only. In order to set the value for a socket directly in the GUI, one can add a property to it. A property is the data that can be displayed/edited in the GUI directly, which is usually a simple data type, such as int, string, boolean, etc.

### Property

A socket can has a property. The data of the property will be used when there is no connection to the input port. The property can be added when define a custom port. Or it can be added later by using ``add_property`` method.

In `aiida-workgraph`, all socket must have a property. The value of the property will be used when there is no connection to the input port.

In [10]:
def create_sockets(self):
   # create a General port.
   inp = self.inputs.new("General", "symbols")
   # add a string property to the port with default value "H".
   inp.add_property("String", "default", default="H")

### Serialization
If you use non-AiiDA data as inputs/outputs of a `Normal` node, the data type of the socket will also indicate how to serialize data and deserialize the data.
